In [1]:
import pandas as pd
from ast import literal_eval
import json

In [2]:
df = pd.read_csv('QASet.csv')

In [3]:
answer_df = pd.read_csv('AnswerSet.csv')

In [4]:
def dfParser(str):
    lst = literal_eval(str)
    rule_df = pd.DataFrame(lst[0])
    return rule_df

In [5]:
jdf = dfParser(df.iloc[6]['judgement'])

In [6]:
df.iloc[6]['judgement']

'[{"answer":["A","B"],"rank":[1,2],"response":["C","D"]}]'

In [7]:
jdf

,answer,rank,response
0,A,1,C
1,B,2,D


In [8]:
answer = 'A'
rank = jdf[jdf.answer==answer].loc[0]['rank']

In [9]:
rank

1

In [10]:
def check(check_rule,answer):
    current_answer = answer
    if check_rule == "null":
        return current_answer
    else:
        rule_elements = check_rule.split(',')
        last_answer = answer_df[answer_df.sub_id==rule_elements[0]]['value'].to_list()[0]
        print(last_answer)
        if rule_elements[2] == "sum":
            if rule_elements[3] == "=":
                if int(last_answer) + int(current_answer) == int(rule_elements[4]):
                    return current_answer
                else:
                    return "retry"
            else:
                if int(last_answer) - int(current_answer) == int(rule_elements[4]):
                    return current_answer
                else:
                    return "retry"
        else:
            return "retry"

In [11]:
def jump_to(raw_jump,final_answer,index):
    if len(raw_jump) == 5:
        distance = df[df.sub_id == raw_jump].index.to_list()[0] - index
        index = index + distance
        return index
    elif raw_jump == "end":
        return 0
    else:
        if final_answer == "retry":
            return index
        else:
            jdf = pd.DataFrame(dfParser(raw_jump))
            jump_to = jdf[jdf.answer==final_answer].iloc[0]['jump']
            distance = df[df.sub_id == jump_to].index.to_list()[0] - index
            index = index + distance
            return index

In [12]:
def judge(qa_row,answer,index):
    raw_judgement = qa_row[4]
    raw_rule = qa_row[5]
    raw_jump = qa_row[6]
    judgement_df = dfParser(raw_judgement)
    rule_df = dfParser(raw_rule)
    if rule_df.iloc[0][nlp]:
        #需要nlp，則把answer用nlp的回傳值覆蓋
        answer = nlp(answer)
        #使用check()，處理最終answer。通過檢查，answer值不變。不然就一定是error，提醒程式準備進行重複操作
        final_answer = check(rule_df.iloc[0]['check'],answer)
        #取得rank，準備寫入
        rank = judgement_df[judgement_df.answer==final_answer].iloc[0]['rank']
        #查詢對應的response
        response = judgement_df[judgement_df.answer==final_answer].iloc[0]['response']
        #計算跳轉所需的index
        idx = jump_to(raw_jump,final_answer,index)
    else:
        #使用check()，處理最終answer。通過檢查，answer值不變。不然就一定是error，提醒程式準備進行重複操作
        final_answer = check(rule_df.iloc[0]['check'],answer)
        #取得rank，準備寫入
        rank = judgement_df[judgement_df.answer==final_answer].iloc[0]['rank']
        #查詢對應的response
        response = judgement_df[judgement_df.answer==final_answer].iloc[0]['response']
        #計算跳轉所需的index
        idx = jump_to(raw_jump,final_answer,index)
    return response,idx

In [13]:
def nlp(a):
    return 0

In [15]:
index = 0
while index < len(df):
    #第一步，讀取問題，Script做
    question = df.iloc[index]['question']
    #第二步，拋出問題，Script做
    print(question)
    #第三步，接收問題，Judge做
    user_answer = input()
    #loc法增刪改查
    answer_df.loc[index,'value'] = user_answer
    #向內傳入index
    response, idx = judge(list(df.iloc[index]),user_answer,index)
    #第四步，回傳index值
    print(response)
    if idx == 0:
        break
    else:
        index = idx

第一題
A
C
第二題
30
C
第三題
50
30


IndexError: single positional indexer is out-of-bounds

In [ ]:
#rank需要包含所有情況，也需要考慮是否給一個有問題的結果賦予rank